**Цель работы:** получить способности и навыки построения алгоритмов систем технического зрения, предназначенных для управления автономными мобильными роботизированными объектами.

**Задание:** Реализовать алгоритм, при котором мобильный робот `TurtleBro` подъезжает к размеченной красной линии перед светофором, после чего в зависимости от сигнала светофора либо продолжает движение либо ожидает разрешения начать движение.

`TurtleBro` – учебно-методический комплекс, включающий программную и аппаратную платформу для изучения `Robot Operation System (ROS)`, OС Linux и принципов разработки современной робототехники.

`Robot Operating System (ROS)` - это гибкая платформа (фреймворк) для разработки программного обеспечения роботов. Это набор разнообразных инструментов, библиотек и определенных правил, целью которых является упрощение задач разработки ПО роботов.

**Основные термины**

`Мастер (Master), Мастер-Нода`

Мастер выполняет роль сервера имен для возможности подключения между собой различных нод. Команда roscore запускает сервер мастера, и после этого к нему могут подключиться и зарегистрироваться ноды ROS. Связь между нодами (обмен сообщениями), невозможна без запущенного мастера.

При запуске ROS roscore, мастер будет запущен по адресу URI, установленным в переменной окружения ROS_MASTER_URI. По умолчанию адрес использует IP-адрес локального ПК и номер порта 11311

`Нода (Node)`

Понятие ноды, относится к наименьшей "рабочей" единицы используемой в ROS. Можно провести аналогию с одной исполняемой программой. ROS рекомендует создать одну ноду для каждой задачи, что позволить легче использовать ее в других проектах.

При запуске нода регистрирует информацию о себе на мастере (название ноды, типы обрабатываемых сообщений). Зарегистрированная нода может взаимодействовать с другими нодами (получать и отправлять запросы). Важно отметить что обмен сообщениями между нодами работает без участия мастера (соединение между нодами происходит на прямую). Мастер обеспечивает только единое пространство имен для решения вопроса куда подключиться к конкретной ноде. Адрес запуска ноды, берётся из переменной окружения ROS_HOSTNAME, которая должна быть определенна до запуска. Порт устанавливается на произвольное уникальное значение.

`Сообщение (Message)`

Ноды отправляют и принимают данные между собой, согласно заданного формата. Эти данные называют Сообщения, а описание Типом Сообщения.

Сообщения могут быть как простых типов (integer, float, boolean), так и могут состоять из сложных структур, содержащих вложенные сообщения и массивы сообщений).

Например для сообщения с координатами обьекта (XYZ) есть существующий тип сообщения `geometry_msgs/Point.msg` который описывается:
```Python
float64 x
float64 y
float64 z
```
`Топик (Topic), модель Издатель и Подписчик`

Топик (Topic), это один из видов обмена сообщениями, который буквально похож на тему в разговоре. Нода издателя (publisher) сначала регистрирует свою тему на мастере, а затем начинает публикацию сообщений в эту тему (топик). Узлы подписчиков, которые хотят получать информацию из этой темы (топика) при помощи мастера получают адрес этой темы и далее получают сообщения из этого топика.

`Издатель (Publisher)`

Издателем называется процесс, который рассылает сообщения в рамках созданного топика для других нод. Одна нода может содержать несколько издателей, публикующих данные в разные топики.

`Подписчик (Subscriber)`

Подписчиком (Subscriber) называется процесс, который получает сообщения из определенного топика (Topic). Подписчик (Subscriber) регистрируется на Мастере (Master), указывая какие топики Подписчик (Subscriber) хочет получать. После этого Издатель (Publisher) начинает отправлять сообщения подписчику. Связь с топиком для подписчика является асинхронной (издатель публикует сообщения в не зависимости от статуса подписчиков). Этот тип взаимодействия удобно применять для работы с датчиками, которые непрерывно передают полученные значения.

**Работа с Topic**

Модель работы в режиме Topic подразумевает использование одного типа сообщения как для Издателя (Publisher) и подписчика (Subscriber).

Модель Topic являются однонаправленной и подразумевает непрерывную отправку или получение сообщений. Такой способ коммуникации подходит для датчиков, которым требуются периодическая передача данных. Несколько подписчиков могут получать сообщения от одного издателя и наоборот (возможна работа несколько издателей).

На изображении ниже показана модель работы датчика температуры, когда его данные получают различные ноды.
<img src="http://docs.voltbro.ru/starting-ros/.gitbook/assets/ros_topic.png" title=""/>

**Публикация сообщений**

Для работы с топиками мы будем использовать библиотеку `rospy`.
Вы можете создать обработчик для публикации сообщения в Топик с помощью класса `rospy.Publisher`:
```Python
pub = rospy.Publisher('topic_name', std_msgs.msg.String, queue_size=10)
```
После инициализации вы можете публиковать сообщения:
```Python
pub.publish(std_msgs.msg.String("foo"))
```
В данном примере:

`topic_name` - Название топика для публикации сообщений
`std_msgs.msg.String` - Тип передаваемого сообщения
`"foo"` - Переданное сообщение
`"queue_size"` - Длина очереди
Полный пример кода Издателя (publisher):
```Python
import rospy
from std_msgs.msg import String

pub = rospy.Publisher('hello', String, queue_size=10)
rospy.init_node('hello_topic_publisher')


while not rospy.is_shutdown():
    pub.publish("Hello World")
    rospy.sleep(1)
```
**Прием сообщений**

Для приема сообщения необходимо воспользоваться rospy.Subscriber. 
Пример реализации Подписчика (Subscriber):
```Python
import rospy
from std_msgs.msg import String

def callback(data):
    rospy.loginfo("I heard %s",data.data)

def subscriber():
    rospy.init_node('hello_topic_subscriber')
    rospy.Subscriber("hello", String, callback)
    # spin() simply keeps python from exiting until this node is stopped
    rospy.spin()

if __name__ == '__main__':
    subscriber()
```
На первый аргумент функции `rospy.Subscriber` подаётся название темы (Topic), из которой будет осуществляться получение информации. Вторым аргументом является тип сообщения. Третий аргумент это используемая функция (метод). Когда новое сообщение будет получено, будет вызван заданный метод с сообщением в качестве первого аргумента.

Рассматриваемая сцена выглядит следующим образом:

<img src="https://sun9-59.userapi.com/impg/SpIb18xdIlck1mTrPTd3Mm9r5XqiOVS7xNRpuA/eQRfEIzwd4w.jpg?size=640x480&quality=96&sign=ab6f397c8421c453408e08f928741365&type=album" title=""/>

Для получения кадров с камеры робота необходимо использовать следующий код:

```Python
import rospy
import cv2
import numpy as np
from sensor_msgs.msg import CompressedImage

rospy.init_node(Имя узла) # задаём имя вашего узла в ковычках

def cb_video_capture(image_msg):
    global image_from_ros_camera
    np_arr = np.frombuffer(image_msg.data, np.uint8)
    image_from_ros_camera = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    rospy.sleep(0.03)

rospy.Subscriber("/front_camera/image_raw/compressed", CompressedImage, cb_video_capture)
```

Вы можете импортировать стандартные библиотеки Python, такие как sys и time. Также вы должны импортировать библиотеку rospy для того, чтобы реализовать ROS узел. Импорт sensor_msgs.msg объясняется тем, что теперь в вашем коде возможно использование данного типа сообщения.


Строчка, `rospy.init_node(NAME)`, очень важна, так как она сообщает rospy имя вашего узла — пока rospy не имеет этой информации, он не может связаться с Мастером ROS (ROS Master). 

Таким образом изображение полученое с камеры робота будет храниться в переменной `image_from_ros_camera` в виде `numpy` массива. Для того чтобы сохранить полученное изображение можно воспользоваться следующей функцией:
```Python
cv2.imwrite("/home/pi/JPG.jpg", image_from_ros_camera)
```

Для управления роботом осуществляется публикация сообщений о его скорости в топик `cmd_vel`. Тип необходимого сообщения загружается из библиотеки `from geometry_msgs.msg import Twist`. Перед тем как публиковать сообщение его сначала нужно сформировать с помощью соответствующей переменной:
```Python
vel = Twist()
vel.linear.x = 0.01
```
После чего осуществляется публикация заданной переменной.

<div class="alert alert-block alert-info">
<b>Обратите внимание:</b> Не устанавливайте скорость робота больше чем значение 0.01. Таким образом вы успеете остановить его (если чтото пойдёт не по плану), также робот не получит повреждения при контакте с объектами на его пути. Для того чтобы остановить робота необходимо отправить сообщение где значение скорости установлено как 0.00</div>

Для реализации паузы между действиями робота используйте команду:
```Python
rospy.sleep(2) # количество секунд бездействия (в данном случае 2 секунды)
```

**Порядок работы**

1. Необходимо определить момент когда робот подъезжает к размеченной линии для того чтобы указать необходимое оставшееся растояние, которое ему нужно проехать до неё. Данный момент характеризуется наличием размеченной линии в нижний части получаемых с камеры кадров сцены.

<img src="https://sun9-59.userapi.com/impg/fPk8o9UCffpSr7uW5a1VVxc2fuhELF4FnSUcew/0UoLB7d_l28.jpg?size=640x480&quality=96&sign=72a43001370b4f08ab3b8fca2559df5d&type=album" title=""/>

Так как изображение представленно в виде numpy массива, то можно взять необходимый срез данного массива, чтобы рассмотреть определённую часть изображения. Пример:
```Python
import matplotlib.pyplot as plt

img = plt.imread('/home/pi/JPG.jpg')
img[100:150,10:60]
```
В данном примере в квадратных скобках задаются индексы элементов массива. Так как массив имеет 3 измерения, через запятую сперва задаётся значение строки массива, затем значение столбца, после чего значение канала изображения. Для того чтобы работать не с одним элементом массива а с некоторым их количеством можно задать с какого индекса по какой будут взяты элементы. 

В примере там где строка `img[100:150,10:60]` индексы слева от запятой соответствуют строкам с 100 (включительно) до 150 (не включительно), для взятия среза элементов массива используется знак `:`. Аналогично справа от запятой указываются индексы столбцов массива, с 10 (включительно) до 60 (не включительно).

2. После того как робот занимает своё положение перед размеченной линией ему необходимо определить сигнал светофора, после чего в соответствии с сигналом либо продолжать движение в течении нескольких секунд, а затем остановиться и завершить программу. В ином случае оставаться на месте и ожидать сигнала светофора, разрешающего движение.

Для распознавания сигнала светофора, необходимо определить его цвет. Для этого можно сохранить изображение сигнала, после чего по индексам рассмотреть значения пикселей соответствующих сигналу. Определив пороги интенсивностей пикселей для каждого канала, необходимо сегментировать области на изображении, соответствующие заданным порогам. После чего если данные области имеют размер сигнала светофора, запускаются команды управления роботом. 


Так как изображения полученные с камеры робота представляют собой numpy массив, то для обработки изображения могут быть полезны следующие операции над массивами:

**Определение размера массива** (высота, ширина, количество каналов)
```Python
img.shape
```
**Добавление измерения к массиву**

```Python
img[..., None]
```

**Сумма значений элементов массива** (в скобочках можно указать по какому измерению осуществлять операцию)
```Python
img.sum()
img.sum(1)
```
**Определение минимального значения массива**

```Python
img.min()
img.min(1)
```

**Определение максимального значения массива**

```Python
img.max()
img.max(1)
```

**Расчёт среднего значения массива**

```Python
img.mean()
img.mean(1)
```

**Объединение нескольких массивов по выбранному измерению**


```Python
np.concatenate((img1, img2, img3, img4), 2)
```

**Проверка каждого элемента массива на соответствие условию** (Формирования массива такого же размера, где каждый элемент принимает значение True если он попадает под условие, в ином случае значение False)

```Python
img>128
img==128
img<128
```
**Реализация операции *И* для элементов массива по выбранному измерению** 

```Python
np.all(img, axis=2)
```

**Реализация операции *ИЛИ* для элементов массива по выбранному измерению** 

```Python
np.any(img, axis=2)
```

**Построение нового массива, в котором значения зависят от того выполняется ли заданное условие** (Первый аргумент это само условие, второй - чему будут равны значения попадающие под заданное условие, третий аргумент - чему будут равны значения не попадающие под заданное условие)

```Python
np.where(img>128,img,0)
np.where(img>128,47,16)
```

<div class="alert alert-block alert-info">
<b>Обратите внимание:</b> Массив изображения, полученный при открытии сохранённой методом «cv2.imwrite» картинки, отличается от массивов изображений, полученных с камеры робота. Таким образом если в первом массиве каналы представляют собой последовательность (R,G,B), то во втором случае последовательность каналов формируется как (B,G,R). Также значения интенсивности пикселей может немного отличаться</div>